## Test for interventions

In [ ]:
from snowplow_signals import Signals
from dotenv import load_dotenv
import os

load_dotenv()

sp_signals = Signals(
    api_url=os.environ["SP_API_URL"],
    api_key=os.environ["SP_API_KEY"],
    api_key_id=os.environ["SP_API_KEY_ID"],
    org_id=os.environ["SP_ORG_ID"],
)

### Define a new attribute

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Event,
    Criterion,
    Criteria,
    EventProperty,
    EntityProperty,
)

count_product_views = Attribute(
    # TODO
)

count_add_to_cart = Attribute(
    # TODO
)

total_cart_value = Attribute(
    # TODO
)


### Wrapping the attribute in a group

All features need to be included in feature views that can be considered as "tables" of features.

Attribute groups are immutable and versioned.

In [ ]:
from snowplow_signals import StreamView, domain_userid

view = StreamView(
    name="ecom_attributes",
    # TODO
)

### Testing the view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
sp_signals.test(view=view)

### Wrap the attribute group in a service

In [ ]:
from snowplow_signals import Service

service = Service(
    # TODO
)

### Publish the attribute group and service

Pulbish the attribute group to start processing it.

In [ ]:
sp_signals.apply([view, service])

### Retrieve attributes for the service

In [ ]:
service.get_attributes(
    signals=sp_signals,
    entity="domain_userid",
    identifier="be240d67-f4bf-4f66-9b19-0338a31c1c17",
)

### Create interventions

In [ ]:
from snowplow_signals import (
    RuleIntervention,
    InterventionCriterion,
)

cart_abandonment_intervention = RuleIntervention(
    # TODO
)

discount_intervention = RuleIntervention(
    # TODO
)

free_shipping_intervention = RuleIntervention(
    # TODO
)


### Apply the intervention

In [ ]:
sp_signals.apply(
    [
        cart_abandonment_intervention,
        discount_intervention,
        free_shipping_intervention,
    ]
)
